In [1358]:
import numpy as np
import math
import random

p = 26570
x = 0
y = 0
z = 6370
d = .0001
q = 299792.458
eps = 1e-8
epsD = eps*q

theta = np.array([math.pi/2*random.random(),math.pi/2*random.random(),math.pi/2*random.random(),math.pi/2*random.random()])
phi = np.array([math.pi*2*random.random(),math.pi*2*random.random(),math.pi*2*random.random(),math.pi*2*random.random()])

def A(x):
    return p*np.cos(phi[x])*np.cos(theta[x])
def B(x):
    return p*np.cos(phi[x])*np.sin(theta[x])
def C(x):
    return p*np.sin(phi[x])
a = np.array([A(0),A(1),A(2),A(3)])
b = np.array([B(0),B(1),B(2),B(3)])
c = np.array([C(0),C(1),C(2),C(3)])

satellite0 = np.array([a[0],b[0],c[0]])
satellite1 = np.array([a[1],b[1],c[1]])
satellite2 = np.array([a[2],b[2],c[2]])
satellite3 = np.array([a[3],b[3],c[3]])
satellitePositions = np.array([satellite0,satellite1,satellite2,satellite3])

def R(x):
    return math.sqrt(a[x]**2+b[x]**2+(c[x]-6370)**2)
satelliteRanges = np.array([R(0),R(1),R(2),R(3)])
r = satelliteRanges

def T(x):
    return d + r[x]/q
t = np.array([T(0),T(1),T(2),T(3)])

print(satellitePositions,'\n',r,'\n',t)

[[ -2543.84649944  -6502.96301221  25636.01406321]
 [-20053.05305117  -2986.48605917  17173.26015483]
 [ -1328.37150108  -1808.44786388 -26475.07970675]
 [-15226.78904707  -9667.38712042  19510.29014597]] 
 [20492.41276265 22972.90720887 32921.63901546 22315.48125272] 
 [0.06845533 0.07672937 0.10991477 0.07453643]


In [1359]:
def Newton_system(F, DF, x0, t, N=1000, e=1e-7):
    x=x0
    #print(x)
    F_value = F(x,satellitePositions,t)
    '''print(F_value)
    print(DF(x,satellitePositions,t))
    print(satellitePositions,"\n",t)/'''
    F_norm = np.linalg.norm(F_value, ord=2)  # l2 norm of vector
    steps = 0
    while abs(F_norm) > e and steps < N:
        s = np.linalg.solve(DF(x,satellitePositions,t), F_value)
        x = x - s
        F_value = F(x,satellitePositions,t)
        F_norm = np.linalg.norm(F_value, ord=2)
        steps = steps + 1
        #print(x)
    # Either a solution is found, or too many iterations
    if abs(F_norm) < e:
        steps = steps-1
        print('The receiver is located at approximately (',x[0],',',x[1],',',x[2],').')
        print('The time drift is approximately',x[3],'seconds.')
        print('Answer',x,'reached in',steps,'steps.')
        print('Our answer is a little off, since F(v) does not equal zero, but rather',F(x,satellitePositions,t),'.')
    else:
        print('The sequence diverges.')

In [1360]:
v0 = np.array([0,0,6370,0])
def F(v,sp,t):
    return np.array([
        (v[0]-sp[0][0])**2 + (v[1]-sp[0][1])**2 + (v[2]-sp[0][2])**2 - q**2*(v[3]-t[0])**2,
        (v[0]-sp[1][0])**2 + (v[1]-sp[1][1])**2 + (v[2]-sp[1][2])**2 - q**2*(v[3]-t[1])**2,
        (v[0]-sp[2][0])**2 + (v[1]-sp[2][1])**2 + (v[2]-sp[2][2])**2 - q**2*(v[3]-t[2])**2,
        (v[0]-sp[3][0])**2 + (v[1]-sp[3][1])**2 + (v[2]-sp[3][2])**2 - q**2*(v[3]-t[3])**2
    ])

def DF(v,sp,t):
    return np.array([
        [(v[0]-sp[0][0])*2,(v[1]-sp[0][1])*2,(v[2]-sp[0][2])*2,-q**2*(v[3]-t[0])*2],
        [(v[0]-sp[1][0])*2,(v[1]-sp[1][1])*2,(v[2]-sp[1][2])*2,-q**2*(v[3]-t[1])*2],
        [(v[0]-sp[2][0])*2,(v[1]-sp[2][1])*2,(v[2]-sp[2][2])*2,-q**2*(v[3]-t[2])*2],
        [(v[0]-sp[3][0])*2,(v[1]-sp[3][1])*2,(v[2]-sp[3][2])*2,-q**2*(v[3]-t[3])*2]
    ])

Newton_system(F,DF,v0,t)

The receiver is located at approximately ( 5.010604066998452e-13 , -9.158852767558239e-12 , 6369.999999999998 ).
The time drift is approximately 0.00010000000000000509 seconds.
Answer [ 5.01060407e-13 -9.15885277e-12  6.37000000e+03  1.00000000e-04] reached in 12 steps.
Our answer is a little off, since F(v) does not equal zero, but rather [-5.96046448e-08  0.00000000e+00  0.00000000e+00  5.96046448e-08] .


In [1361]:
def Newton_return(F, DF, x0, t, N=100, e=1e-7):
    x=x0
    F_value = F(x,satellitePositions,t)
    F_norm = np.linalg.norm(F_value, ord=2)  # l2 norm of vector
    steps = 0
    while abs(F_norm) > e and steps < N:
        s = np.linalg.solve(DF(x,satellitePositions,t), F_value)
        x = x - s
        F_value = F(x,satellitePositions,t)
        F_norm = np.linalg.norm(F_value, ord=2)
        steps = steps+1
    if abs(F_norm) < e:
        return x
    else:
        return np.array([-1e+12])

In [1362]:
def EMNFinder(deltaT = [(-1)**random.randint(1,2)*eps,(-1)**random.randint(1,2)*eps,(-1)**random.randint(1,2)*eps,(-1)**random.randint(1,2)*eps]):
    tBar = t + deltaT 
    if Newton_return(F,DF,v0,tBar)[0] < -1e+11:
        return -1
    else:
        xBar = Newton_return(F,DF,v0,tBar)[0]
        yBar = Newton_return(F,DF,v0,tBar)[1]
        zBar = Newton_return(F,DF,v0,tBar)[2]
        dBar = Newton_return(F,DF,v0,tBar)[3]
        deltaX = abs(x-xBar)
        deltaY = abs(y-yBar)
        deltaZ = abs(z-zBar)
        deltaD = abs(d-dBar)
        EMN = max(deltaX,deltaY,deltaZ)/epsD
        return EMN

print('The Error Magnification Number is ',EMNFinder())

The Error Magnification Number is  -1


In [1368]:
#max of 50 trials
maxEMN = max(
    EMNFinder([eps,eps,eps,eps]),
    EMNFinder([eps,eps,eps,-eps]),EMNFinder([eps,eps,-eps,eps]),EMNFinder([eps,-eps,eps,eps]),EMNFinder([-eps,eps,eps,eps]),
    EMNFinder([eps,eps,-eps,-eps]),EMNFinder([eps,-eps,eps,-eps]),EMNFinder([-eps,eps,eps,-eps]),EMNFinder([eps,-eps,-eps,eps]),EMNFinder([-eps,eps,-eps,eps]),EMNFinder([-eps,-eps,eps,eps]),
    EMNFinder([eps,-eps,-eps,-eps]),EMNFinder([-eps,eps,-eps,-eps]),EMNFinder([-eps,-eps,eps,-eps]),EMNFinder([-eps,-eps,-eps,eps]),
    EMNFinder([-eps,-eps,-eps,-eps])
    )
print('Max EMN:',maxEMN)
print('Max position error:',maxEMN*epsD,'meters')

print('Inputs theta and phi:\n',theta*180/math.pi,'\n',phi*180/math.pi)

Max EMN: 7.822250078837552
Max position error: 0.023450515782254034 meters
Inputs theta and phi:
 [68.63542337  8.47075648 53.70134837 32.41119247] 
 [105.2367552  139.73376347 265.15547987 132.75209324]
